# LogisticClassification

Binary Classification : 두가지로 분류  
대표적 예 : 스펨 메일 분류, 페이스북 알고리즘, StockPricing 예측 등

Linear Regression가 분류에서 가지는 한계  
* 특잇값 때문에 왜곡이 일어날 수 있음  
* 0,1 외에 너무 큰 값을 반환할 수 있음

따라서 0,1사이의 값을 주는 함수를 찾음  

 시그모이드 $$ g(z) = \frac{1}{1+ e^{-z}} $$

이제 Hypothesis를  
$ z = WX $  
$ H(x) = g(z) $  
즉, $$ H(X) = \frac{1}{1+ e^{-W^TX}} $$

기존 손실함수는  
$$ Cost = \frac{1}{m} \sum_{i=1}^m (H(x_i) - y_i)^2) $$  
이때 이를 바꾼 $H(x)$에 대해 그대로 적용하면 Convex하지 않고 울퉁불퉁한 그래프 모양이 나옴  
이때 Local minimum이 많이 생기기 때문에 Global minimun으로 학습이 되지 않음  
따라서 다른 손실 함수 필요

**New cost function for logistic**
$$ cost(W) = \frac{1}{m} \sum c(H(x),y) $$
$$c(H(x), y) = \begin{cases} 
-\log(H(x)) & y=1 \\ 
-\log(1-H(x)) & y=0 
\end{cases}$$

**해석**  
자연상수e가 함수 모양을 울퉁불퉁하게 만들기에 log를 활용하여 매끄럽게 바꿔줌  
실제레이블이 1일때
* 예측값이 1로 맞았을때는 cost값이 0  
* 예측값이 0으로 틀렸을때는 cost값이 무한히 발산  

실제레이블이 0일때
* 예측값이 0으로 맞았을때는 cost값이 0  
* 예측값이 1로 틀렸을때는 cost값이 무한히 발산

이는 case로 쪼개져 있으므로 손실함수로 프로그램에 사용하기 위해 한줄로 표현시  
$$ cost(W) = \frac{1}{m} \sum c(H(x),y) $$
$$c(H(x), y) = -y\log(H(x)) -(1-y)\log(1-H(x))$$